In [1]:
from __future__ import print_function
import os
import warnings
warnings.filterwarnings('ignore')
import torch, pdb
import pickle
import argparse
import numpy as np
from m2det import build_net
from utils.timer import Timer
import torch.backends.cudnn as cudnn
from layers.functions import Detect,PriorBox
from data import BaseTransform
from configs.CC import Config
from tqdm import tqdm
from utils.core import *
from torchsummary import summary


print_info('----------------------------------------------------------------------\n'
           '|                       M2Det Evaluation Program                     |\n'
           '----------------------------------------------------------------------', ['yellow','bold'])
global cfg
cfg = Config.fromfile("configs/m2det512_mobilenetv3.py")
trained_model='weights/mbv3.pth'
# trained_model='../M2Det_196/weights/3.pth'

anchor_config = anchors(cfg)
if not os.path.exists(cfg.test_cfg.save_folder):
    os.mkdir(cfg.test_cfg.save_folder)
anchor_config = anchors(cfg)
print_info('The Anchor info: \n{}'.format(anchor_config))
priorbox = PriorBox(anchor_config)
with torch.no_grad():
    priors = priorbox.forward()
    if cfg.test_cfg.cuda:
        priors = priors.cuda()



#     trained_model='../M2Det_196/weights/M2Det_VOC_size512_netresnet50_epoch29.pth'

net = build_net('test',
                size = cfg.model.input_size,
                config = cfg.model.m2det_config)
init_net(net, cfg, trained_model)
print_info('===> Finished constructing and loading model',['yellow','bold'])
net.eval()
_set = 'eval_sets'
testset = get_dataloader(cfg, 'VOC', _set)
if cfg.test_cfg.cuda:
    net = net.cuda()
    cudnn.benchmark = True
else:
    net = net.cpu()
detector = Detect(cfg.model.m2det_config.num_classes, cfg.loss.bkg_label, anchor_config)
save_folder = os.path.join(cfg.test_cfg.save_folder, '')
_preprocess = BaseTransform(cfg.model.input_size, cfg.model.rgb_means, (2, 0, 1))
cuda = cfg.test_cfg.cuda
transform = _preprocess
max_per_image = cfg.test_cfg.topk
thresh = cfg.test_cfg.score_threshold

########################


if not os.path.exists(save_folder):
    os.mkdir(save_folder)

num_images = len(testset)
print_info('=> Total {} images to test.'.format(num_images),['yellow','bold'])
num_classes = cfg.model.m2det_config.num_classes
all_boxes = [[[] for _ in range(num_images)] for _ in range(num_classes)]
import pdb
# pdb.set_trace()
summary(net, input_size=(3, 320, 320))

_t = {'im_detect': Timer(), 'misc': Timer()}
det_file = os.path.join(save_folder, 'detections.pkl')
tot_detect_time, tot_nms_time = 0, 0
print_info('Begin to evaluate',['yellow','bold'])
for i in tqdm(range(num_images)):
    img = testset.pull_image(i)
    # step1: CNN detection
    _t['im_detect'].tic()
    boxes, scores = image_forward(img, net, cuda, priors, detector, transform)
    inds = np.where(scores > cfg.test_cfg.score_threshold)[0]
    if len(inds) == 0:
        continue
    boxes = boxes[inds]
    scores = scores[inds]

    detect_time = _t['im_detect'].toc()
    # step2: Post-process: NMS
    _t['misc'].tic()
    nms_process(num_classes, i, scores, boxes, cfg, 0.00, all_boxes, max_per_image)
    nms_time = _t['misc'].toc()

    tot_detect_time += detect_time if i > 0 else 0
    tot_nms_time += nms_time if i > 0 else 0

with open(det_file, 'wb') as f:
    pickle.dump(all_boxes, f, pickle.HIGHEST_PROTOCOL)
with open(det_file, 'rb') as f:
    all_boxes = pickle.load(f)
print_info('===> Evaluating detections',['yellow','bold'])
testset.evaluate_detections(all_boxes, save_folder)
print_info('Detect time per image: {:.3f}s'.format(tot_detect_time / (num_images-1)))
print_info('Nms time per image: {:.3f}s'.format(tot_nms_time / (num_images - 1)))
print_info('Total time per image: {:.3f}s'.format((tot_detect_time + tot_nms_time) / (num_images - 1)))
print_info('FPS: {:.3f} fps'.format((num_images - 1) / (tot_detect_time + tot_nms_time)))

----------------------------------------------------------------------
|                       M2Det Evaluation Program                     |
----------------------------------------------------------------------
The Anchor info: 
{'feature_maps': [64, 32, 16, 8, 4, 2], 'min_dim': 512, 'steps': [8, 16, 32, 64, 107, 320], 'min_sizes': [40.96, 76.8, 168.96, 261.12, 353.28, 445.44], 'max_sizes': [76.8, 168.96, 261.12, 353.28, 445.44, 537.6], 'aspect_ratios': [[2, 3], [2, 3], [2, 3], [2, 3], [2, 3], [2, 3]], 'variance': [0.1, 0.2], 'clip': True}
===> Constructing M2Det model
Loading resume network...
===> Finished constructing and loading model
=> Total 8901 images to test.


  0%|          | 0/8901 [00:00<?, ?it/s]

----------------------------------------------------------------
        Layer (type)                Output size               Kernel Size         Param #            GOPs
            Conv2d-1    [-1, 16, 160, 160]               [3, 3]             432      22,528,000
            Hswish-3    [-1, 16, 160, 160]               [0, 0]               0               0
            Conv2d-4    [-1, 16, 160, 160]               [1, 1]             256      13,516,800
            Conv2d-7    [-1, 16, 160, 160]               [3, 3]             144     118,374,400
          Identity-9    [-1, 16, 160, 160]               [0, 0]               0               0
           Conv2d-11    [-1, 16, 160, 160]               [1, 1]             256      13,516,800
 MobileBottleneck-13    [-1, 16, 160, 160]               [0, 0]               0               0
           Conv2d-14    [-1, 64, 160, 160]               [1, 1]           1,024      54,067,200
           Conv2d-17    [-1, 64, 160, 160]               [3, 

100%|██████████| 8901/8901 [31:15<00:00,  4.79it/s]


===> Evaluating detections
Writing vehicle VOC results file
Writing pedestrian VOC results file
Writing rider VOC results file
VOC07 metric? No
Saving cached annotations to /ssd2/CarRecog/creDa_train/annotations_cache/annots.pkl
AP for vehicle = 0.9231
Saving cached annotations to /ssd2/CarRecog/creDa_train/annotations_cache/annots.pkl
AP for pedestrian = 0.7945
Saving cached annotations to /ssd2/CarRecog/creDa_train/annotations_cache/annots.pkl
AP for rider = 0.8561
Mean AP = 0.8579
~~~~~~~~
Results:
0.923
0.794
0.856
0.858
~~~~~~~~

--------------------------------------------------------------
Results computed with the **unofficial** Python eval code.
Results should be very close to the official MATLAB eval code.
Recompute with `./tools/reval.py --matlab ...` for your paper.
-- Thanks, The Management
--------------------------------------------------------------
Detect time per image: 0.046s
Nms time per image: 0.158s
Total time per image: 0.204s
FPS: 4.910 fps
